In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tqdm

### dataset 

In [ ]:
data = load_wine(as_frame=True)

pandas_df = data['data']
targets= data['target']

display(pandas_df.head())
display(targets)

In [ ]:
scaler = StandardScaler()
df = scaler.fit_transform(pandas_df.to_numpy())

X_train, X_test, y_train, y_test = train_test_split(df, targets, test_size=0.3, shuffle=True)

In [ ]:
tensor_X_train = torch.from_numpy(X_train).to(torch.float32)
tensor_y_train = torch.from_numpy(y_train.to_numpy()).to(torch.int8)

display(tensor_X_train)
display(tensor_X_train.shape)
display(tensor_y_train)
display(tensor_y_train.shape)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_y_train)
for i in train_dataset:
    print(i)
    break

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
for i in train_dataloader:
    print(i)
    break

### model 

In [ ]:
model = nn.Sequential(nn.Linear(13, 32), 
                      nn.ReLU(), 
                      nn.Linear(32, 32),
                      nn.ReLU(), 
                      nn.Linear(32,3))

model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
epochs = 50

for epoch in tqdm.tqdm(range(epochs)):
    for x, y in train_dataloader:
        outputs = model(x)
        tgs = y.to(torch.long)
        
        optimizer.zero_grad()
        loss = criterion(outputs, tgs)
        loss.backward()
        optimizer.step()

In [ ]:
model(train_dataset[0][0].unsqueeze(dim=0))

### eval 

In [ ]:
with torch.no_grad():
    result = model(torch.from_numpy(X_test).to(torch.float32))
    
result[0:5]

In [ ]:
torch.argmax(result, dim=1)

In [ ]:
torch.from_numpy(y_test.to_numpy()).to(torch.int64)

In [ ]:
torch.argmax(result, dim=1) == torch.from_numpy(y_test.to_numpy()).to(torch.int64)